In [ ]:
# start with notebook,
# convert into module later

In [ ]:
from pprint import pprint
from util import generate_problem, verify_witness
INF = 1e9

In [ ]:
def discrete_graph(N):
    mat = [[INF for _ in range(N)] for _ in range(N)]
    for i in range(N):
        mat[i][i] = 0
    return mat

def generate_d_graph(graph):
    N = len(graph)
    E = discrete_graph(N)
    
    for i in range(N):
        E[i][i] = 0 # redundant

    for i in range(N):
        for j in range(N):
            E[i][j] = graph[i][j]

    for k in range(N):
        for i in range(N):
            for j in range(N):
                E[i][j] = min(
                    E[i][j],
                    E[i][k] + E[k][j],
                )

    return E

def consistent(d_graph):
    for i in range(len(d_graph)):
        if d_graph[i][i] < 0: return False
    return True

def get_min_sol(d_graph):
    return [-d_graph[i][0] for i in range(len(d_graph))]

In [ ]:
def pick_constraint_first_unused(available_constraints):
    return list(available_constraints)[0]

def pick_constraint_least_intervals(available_constraints):
    least_intervals = 0
    num = len(available_constraints[least_intervals]['intervals'])
    for idx in range(1, len(available_constraints)):
        if len(available_constraints[idx]['intervals']) < num:
            num = len(available_constraints[idx]['intervals'])
            least_intervals = idx
            
    return available_constraints[least_intervals]

In [136]:
def backtrack(all_constraints, available_constraints, graph, pick_constraint, stats=None, break_on_sat=True):
    if stats == None:
        stats = {'consistent': 0, 'dead': 0, 'total': 0}
        
    if not available_constraints:
        stats['total'] += 1
        d_graph = generate_d_graph(graph)
        cons = consistent(d_graph)
        if cons:
            stats['consistent'] += 1
        else:
            stats['dead'] += 1
        return stats, cons

    constr = pick_constraint(available_constraints)
    available_constraints.remove(constr)
    
    i, j = constr['i'], constr['j']
    for interval in constr['intervals']:
        stats['total'] += 1
        graph[i][j] = interval[1]
        graph[j][i] = -interval[0]

        d_graph = generate_d_graph(graph)
        if consistent(d_graph):
            x = backtrack(all_constraints, available_constraints, graph, pick_constraint, stats, break_on_sat=break_on_sat)
            if break_on_sat and x[1]: return stats, True
        else:
            stats['dead'] += 1
    
        graph[i][j] = INF
        graph[j][i] = INF

    available_constraints.append(constr)
    return stats, False

def solve(num_variables, constraints, pick_constraint, break_on_sat=True):
    return backtrack(
        constraints, 
        constraints,
        discrete_graph(num_variables),
        pick_constraint,
        break_on_sat=break_on_sat
    )

## --- Main Implementation End ---

## --- Preprocessing Implementation ---

Let's test out a few preprocessing algorithms, and below compare how backtrack fares with and without preprocessing

In [171]:
def intersect(I, J):
    """
    intersect intervals given by lists I and J
    note: both lists have to contain ordered disjoint intervals.
    """
    
    # TODO fix this merge lol, experimental
    return cleanup_intervals([
        [max(first[0], second[0]), min(first[1], second[1])] 
          for first in I for second in J  if max(first[0], second[0]) <= min(first[1], second[1])
    ])
    
def compose(I, J):
    """
    compose lists of intervals I and J
    note: make sure they are ordered and disjoint
    """

    return cleanup_intervals([
        (i[0] + j[0], i[1] + j[1])
        for i in I for j in J
    ])

def cleanup_intervals(I):
    """
    given a list of intervals, produce equivalent ordered disjoint intervals
    """
    
    ret = []
    
    for begin, end in sorted(I):
        if ret and ret[-1][1] >= begin - 1:
            ret[-1][1] = max(ret[-1][1], end)
        else:
            ret.append( [begin, end] )
    
    return [(a, b) for a, b in ret]

    
def PC_1(T):
    # i need constraints to be in a specific form to perform random access T_ij
    num_variables = max([max(t['i'], t['j']) for t in T]) + 1
    mat = [ [ list() for i in range(num_variables) ] for j in range(num_variables) ]
    for constr in T:
        i, j, intervals = constr['i'], constr['j'], constr['intervals']
        
        mat[i][j] = intervals
        mat[j][i] = [(-b, -a) for a, b in reversed(intervals)]
        # for reverse, we take each interval (a,b) and convert it into (-b,-a)
        # note we want to also reverse the order of all intervals so that it is again increasing
        
    
    # actual PC-1 core algorithm
    
    # ? S = T
    for k in range(num_variables):
        for i in range(num_variables):
            if i == k: continue
            for j in range(num_variables):
                if i == j: continue
                if k == j: continue
                # T_ij = T_ij + T_ik x T_kj
                print(i, j, mat[i][j])
                mat[i][j] = intersect(mat[i][j], compose(mat[i][k], mat[k][j]))
                print(i, j, mat[i][j])
    # ? until S = T
    
    # convert back to constraint form
    S = []
    print(mat)
    for i in range(num_variables):
        for j in range(i + 1, num_variables):
            if mat[i][j]:
                S += {'i': i, 'j': j, 'intervals': mat[i][j]}
    return S

## --- Preprocessing End ---

In [172]:
num_variables = 15
T = generate_problem(
    variables=num_variables
)

In [173]:
PC_1(T)

1 2 []
1 2 []
1 3 []
1 3 []
1 4 []
1 4 []
1 5 []
1 5 []
1 6 []
1 6 []
1 7 []
1 7 []
1 8 []
1 8 []
1 9 []
1 9 []
1 10 []
1 10 []
1 11 [(-95, -93), (-84, -47), (-47, -40), (21, 30), (46, 56)]
1 11 [(-95, -93), (-84, -40), (21, 30), (46, 56)]
1 12 []
1 12 []
1 13 []
1 13 []
1 14 [(-69, -7)]
1 14 [(-69, -7)]
2 1 []
2 1 []
2 3 []
2 3 []
2 4 [(-40, -19), (83, 92)]
2 4 [(-40, -19), (83, 92)]
2 5 [(-89, -61), (-51, 6), (78, 92)]
2 5 [(-89, -61), (-51, 6), (78, 92)]
2 6 [(-100, -59), (-55, -53), (-52, -51), (-9, 60)]
2 6 [(-100, -59), (-55, -51), (-9, 60)]
2 7 []
2 7 []
2 8 []
2 8 []
2 9 []
2 9 []
2 10 []
2 10 []
2 11 []
2 11 []
2 12 []
2 12 []
2 13 []
2 13 []
2 14 []
2 14 []
3 1 []
3 1 []
3 2 []
3 2 []
3 4 []
3 4 []
3 5 []
3 5 []
3 6 [(-98, -95), (-89, -73), (-52, 31), (37, 97)]
3 6 [(-98, -95), (-89, -73), (-52, 31), (37, 97)]
3 7 [(-71, -60), (-48, 3), (15, 100)]
3 7 [(-71, -60), (-48, 3), (15, 100)]
3 8 []
3 8 []
3 9 []
3 9 []
3 10 []
3 10 []
3 11 []
3 11 []
3 12 []
3 12 []
3 13 []
3 13 []


12 8 [(-74, -58), (-54, -52), (-35, -31), (8, 36)]
12 8 []
12 9 []
12 9 []
12 10 []
12 10 []
12 11 []
12 11 []
12 13 []
12 13 []
12 14 []
12 14 []
13 0 [(-100, 100)]
13 0 []
13 2 []
13 2 []
13 3 []
13 3 []
13 4 []
13 4 []
13 5 [(-95, -14), (-11, 8), (17, 89)]
13 5 []
13 6 []
13 6 []
13 7 []
13 7 []
13 8 []
13 8 []
13 9 []
13 9 []
13 10 [(-74, -72), (-65, -58), (-41, -30), (-17, -14), (13, 61)]
13 10 []
13 11 []
13 11 []
13 12 []
13 12 []
13 14 []
13 14 []
14 0 [(-100, 100)]
14 0 [(-93, 100)]
14 2 []
14 2 []
14 3 [(-50, -23), (-16, 19), (23, 77)]
14 3 []
14 4 [(-65, -63), (-59, -56), (-30, -23), (35, 45), (51, 100)]
14 4 []
14 5 []
14 5 []
14 6 [(-81, -52), (-35, 22), (68, 69)]
14 6 []
14 7 []
14 7 []
14 8 []
14 8 []
14 9 []
14 9 []
14 10 []
14 10 []
14 11 []
14 11 []
14 12 []
14 12 []
14 13 []
14 13 []
0 1 [(-100, 100)]
0 1 []
0 3 []
0 3 []
0 4 []
0 4 []
0 5 []
0 5 []
0 6 []
0 6 []
0 7 []
0 7 []
0 8 []
0 8 []
0 9 []
0 9 []
0 10 []
0 10 []
0 11 [(-100, 100)]
0 11 []
0 12 []
0 12 []
0 13

4 7 []
4 8 []
4 8 []
4 9 []
4 9 []
4 10 []
4 10 []
4 11 []
4 11 []
4 12 []
4 12 []
4 13 []
4 13 []
4 14 []
4 14 []
6 0 []
6 0 []
6 1 []
6 1 []
6 2 []
6 2 []
6 3 []
6 3 []
6 4 []
6 4 []
6 7 []
6 7 []
6 8 []
6 8 []
6 9 []
6 9 []
6 10 []
6 10 []
6 11 []
6 11 []
6 12 []
6 12 []
6 13 []
6 13 []
6 14 []
6 14 []
7 0 []
7 0 []
7 1 []
7 1 []
7 2 []
7 2 []
7 3 []
7 3 []
7 4 []
7 4 []
7 6 []
7 6 []
7 8 []
7 8 []
7 9 []
7 9 []
7 10 []
7 10 []
7 11 []
7 11 []
7 12 []
7 12 []
7 13 []
7 13 []
7 14 []
7 14 []
8 0 []
8 0 []
8 1 []
8 1 []
8 2 []
8 2 []
8 3 []
8 3 []
8 4 []
8 4 []
8 6 []
8 6 []
8 7 []
8 7 []
8 9 []
8 9 []
8 10 []
8 10 []
8 11 []
8 11 []
8 12 []
8 12 []
8 13 []
8 13 []
8 14 []
8 14 []
9 0 []
9 0 []
9 1 []
9 1 []
9 2 []
9 2 []
9 3 []
9 3 []
9 4 []
9 4 []
9 6 []
9 6 []
9 7 []
9 7 []
9 8 []
9 8 []
9 10 []
9 10 []
9 11 []
9 11 []
9 12 []
9 12 []
9 13 []
9 13 []
9 14 []
9 14 []
10 0 []
10 0 []
10 1 []
10 1 []
10 2 []
10 2 []
10 3 []
10 3 []
10 4 []
10 4 []
10 6 []
10 6 []
10 7 []
10 7 []
10 8 

12 6 []
12 6 []
12 7 []
12 7 []
12 9 []
12 9 []
12 10 []
12 10 []
12 11 []
12 11 []
12 13 []
12 13 []
12 14 []
12 14 []
13 0 []
13 0 []
13 1 []
13 1 []
13 2 []
13 2 []
13 3 []
13 3 []
13 4 []
13 4 []
13 5 []
13 5 []
13 6 []
13 6 []
13 7 []
13 7 []
13 9 []
13 9 []
13 10 []
13 10 []
13 11 []
13 11 []
13 12 []
13 12 []
13 14 []
13 14 []
14 0 []
14 0 []
14 1 []
14 1 []
14 2 []
14 2 []
14 3 []
14 3 []
14 4 []
14 4 []
14 5 []
14 5 []
14 6 []
14 6 []
14 7 []
14 7 []
14 9 []
14 9 []
14 10 []
14 10 []
14 11 []
14 11 []
14 12 []
14 12 []
14 13 []
14 13 []
0 1 []
0 1 []
0 2 []
0 2 []
0 3 []
0 3 []
0 4 []
0 4 []
0 5 []
0 5 []
0 6 []
0 6 []
0 7 []
0 7 []
0 8 []
0 8 []
0 10 []
0 10 []
0 11 []
0 11 []
0 12 []
0 12 []
0 13 []
0 13 []
0 14 []
0 14 []
1 0 []
1 0 []
1 2 []
1 2 []
1 3 []
1 3 []
1 4 []
1 4 []
1 5 []
1 5 []
1 6 []
1 6 []
1 7 []
1 7 []
1 8 []
1 8 []
1 10 []
1 10 []
1 11 []
1 11 []
1 12 []
1 12 []
1 13 []
1 13 []
1 14 []
1 14 []
2 0 []
2 0 []
2 1 []
2 1 []
2 3 []
2 3 []
2 4 []
2 4 []
2 5 []
2

4 0 []
4 0 []
4 1 []
4 1 []
4 2 []
4 2 []
4 3 []
4 3 []
4 5 []
4 5 []
4 6 []
4 6 []
4 7 []
4 7 []
4 8 []
4 8 []
4 9 []
4 9 []
4 10 []
4 10 []
4 11 []
4 11 []
4 13 []
4 13 []
4 14 []
4 14 []
5 0 []
5 0 []
5 1 []
5 1 []
5 2 []
5 2 []
5 3 []
5 3 []
5 4 []
5 4 []
5 6 []
5 6 []
5 7 []
5 7 []
5 8 []
5 8 []
5 9 []
5 9 []
5 10 []
5 10 []
5 11 []
5 11 []
5 13 []
5 13 []
5 14 []
5 14 []
6 0 []
6 0 []
6 1 []
6 1 []
6 2 []
6 2 []
6 3 []
6 3 []
6 4 []
6 4 []
6 5 []
6 5 []
6 7 []
6 7 []
6 8 []
6 8 []
6 9 []
6 9 []
6 10 []
6 10 []
6 11 []
6 11 []
6 13 []
6 13 []
6 14 []
6 14 []
7 0 []
7 0 []
7 1 []
7 1 []
7 2 []
7 2 []
7 3 []
7 3 []
7 4 []
7 4 []
7 5 []
7 5 []
7 6 []
7 6 []
7 8 []
7 8 []
7 9 []
7 9 []
7 10 []
7 10 []
7 11 []
7 11 []
7 13 []
7 13 []
7 14 []
7 14 []
8 0 []
8 0 []
8 1 []
8 1 []
8 2 []
8 2 []
8 3 []
8 3 []
8 4 []
8 4 []
8 5 []
8 5 []
8 6 []
8 6 []
8 7 []
8 7 []
8 9 []
8 9 []
8 10 []
8 10 []
8 11 []
8 11 []
8 13 []
8 13 []
8 14 []
8 14 []
9 0 []
9 0 []
9 1 []
9 1 []
9 2 []
9 2 []
9 3 []
9

[]

In [118]:
%%timeit
solve(
    num_variables, 
    T,
    pick_constraint=pick_constraint_least_intervals,
)

KeyboardInterrupt: 

In [ ]:
%%timeit
solve(
    num_variables, 
    T,
    pick_constraint=pick_constraint_first_unused,
)